<a href="https://colab.research.google.com/github/bminixhofer/nnsplit/blob/master/train/evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!git clone https://github.com/bminixhofer/nnsplit

Cloning into 'nnsplit'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 1530 (delta 61), reused 90 (delta 33), pack-reused 1388
Receiving objects: 100% (1530/1530), 12.81 MiB | 28.45 MiB/s, done.
Resolving deltas: 100% (784/784), done.


In [2]:
!pip install -r nnsplit/train/requirements.txt

     |████████████████████████████████| 102kB 2.6MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
     |████████████████████████████████| 829kB 27.0MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 460kB 43.4MB/s 
     |████████████████████████████████| 92kB 9.6MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 307kB 44.1MB/s 
     |████████████████████████████████| 5.5MB 42.4MB/s 
     |████████████████████████████████| 14.6MB 300kB/s 
     |████████████████████████████████| 450kB 43.6MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 71kB 6.7MB/s 
     |████████████████████████████████| 296kB 49.0MB/s 
     |████████████████████████████████| 512kB 46.9MB/s 
     |████████████████████████████████| 276kB 46.8MB/s 
     |███

In [1]:
!wget -O "raw.de.gz" http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.raw.de.gz
!gunzip "raw.de.gz"

!wget -O "raw.en.gz" http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.raw.en.gz
!gunzip "raw.en.gz"

--2020-09-21 20:36:06--  http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.raw.de.gz
Resolving opus.nlpl.eu (opus.nlpl.eu)...193.166.25.9
Connecting to opus.nlpl.eu (opus.nlpl.eu)|193.166.25.9|:80...connected.
HTTP request sent, awaiting response...302 Found
Location: https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/de.txt.gz [following]
--2020-09-21 20:36:06--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/de.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)...86.50.254.19, 86.50.254.18
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.19|:443...connected.
HTTP request sent, awaiting response...200 OK
Length: 475884373 (454M) [application/gzip]
Saving to: ‘raw.de.gz’

raw.de.gz           100%[===================>] 453.84M  2.10MB/s    in 3m 23s  

2020-09-21 20:39:30 (2.23 MB/s) - ‘raw.de.gz’ saved [475884373/475884373]

--2020-09-21 20:39:38--  http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitl

In [2]:
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
     |████████████████████████████████| 12.0 MB 2.3 MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.0-py3-none-any.whl size=12048606 sha256=6e883750227d0cfd981fad1744736468d1da5c4b177af0797cf1b398bc2cca43
  Stored in directory: /tmp/pip-ephem-wheel-cache-v276g5p_/wheels/4a/db/07/94eee4f3a60150464a04160bd0dfe9c8752ab981fe92f16aea
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [5]:
import sys

In [3]:
!pip install nnsplit

     |████████████████████████████████| 1.8 MB 2.0 MB/s 
  Attempting uninstall: nnsplit
    Found existing installation: nnsplit 0.4.9.post0
    Uninstalling nnsplit-0.4.9.post0:
      Successfully uninstalled nnsplit-0.4.9.post0


# Evaluate

In [1]:
import sys
sys.path.append("nnsplit/train")

In [2]:
import evaluate
from evaluate import OpenSubtitlesDataset, Evaluator
from torch.utils.data import Subset
from nnsplit import NNSplit
import spacy
import numpy as np
import torch
import pandas as pd

In [3]:
class NNSplitInterface:
  def __init__(self, splitter):
    self.splitter = splitter

  def split(self, texts):
    out = []
    for split in self.splitter.split(texts):
        out.append([str(x) for x in split])

    return out
  
class SpacyInterface:
  def __init__(self, name, use_sentencizer):
    if use_sentencizer:
      nlp = spacy.load(name, disable=["tagger", "parser", "ner"])
      nlp.add_pipe(nlp.create_pipe("sentencizer"))
    else:
      nlp = spacy.load(name, disable=["tagger", "ner"])

    self.nlp = nlp

  def split(self, texts):
    out = []
    for doc in self.nlp.pipe(texts):
        sentences = []

        for sent in doc.sents:
            sentences.append("".join([x.text + x.whitespace_ for x in sent]))

        out.append(sentences)

    return out

In [4]:
data = [
  [
   "German",
   Subset(OpenSubtitlesDataset("raw.de", 1_000_000), np.arange(100_000)),
   {
       "NNSplit": NNSplitInterface(NNSplit("wandb/run-20200923_145410-3ol591he/model/model.onnx")),
       "Spacy (Tagger)": SpacyInterface("de_core_news_sm", use_sentencizer=False),
       "Spacy (Sentencizer)": SpacyInterface("de_core_news_sm", use_sentencizer=True)
   }
  ],
#   [
#    "English",
#    Subset(OpenSubtitlesDataset("raw.en", 1_000_000), np.arange(100_000)),
#    {
#        "NNSplit": NNSplitInterface(NNSplit.load("en")),
#        "Spacy (Tagger)": SpacyInterface("en_core_web_sm", use_sentencizer=False),
#        "Spacy (Sentencizer)": SpacyInterface("en_core_web_sm", use_sentencizer=True)
#    }
#   ]
]

eval_setups = {
    "Clean": (0.0, 0.0),
    "Partial punctuation": (0.5, 0.0),
    "Partial case": (0.0, 0.5),
    "Partial punctuation and case": (0.5, 0.5),
    "No punctuation and case": (1.0, 1.0),
}

1000000it [00:00, 1499789.57it/s]


In [5]:
results = {}
preds = {}

for dataset_name, dataset, targets in data:
  results[dataset_name] = {}

  for eval_name, (remove_punct_prob, lower_start_prob) in eval_setups.items():
    results[dataset_name][eval_name] = {}
    evaluator = Evaluator(dataset, remove_punct_prob, lower_start_prob)

    for target_name, interface in targets.items():
      correct = evaluator.evaluate(interface.split)
      preds[f"{dataset_name}_{eval_name}_{target_name}"] = {
          "samples": evaluator.texts,
          "correct": correct,
      }
      results[dataset_name][eval_name][target_name] = correct.mean()

100%|██████████| 100000/100000 [00:00<00:00, 174288.53it/s]


In [7]:
pd.DataFrame.from_dict(results["German"]).T

,NNSplit,Spacy (Tagger),Spacy (Sentencizer)
Clean,0.808799,0.833368,0.878471
Partial punctuation,0.528230,0.426458,0.266312
Partial case,0.809457,0.792839,0.876678
Partial punctuation and case,0.500463,0.377201,0.266970
No punctuation and case,0.249335,0.095227,0.007562


In [11]:
pd.DataFrame.from_dict(results["German"]).T

,NNSplit,Spacy (Tagger),Spacy (Sentencizer)
Clean,0.818902,0.833368,0.878471
Partial punctuation,0.463999,0.426458,0.266312
Partial case,0.823565,0.792839,0.876678
Partial punctuation and case,0.447231,0.377201,0.266970
No punctuation and case,0.198165,0.095227,0.007562


In [12]:
pd.DataFrame.from_dict(results["English"]).T

,NNSplit,Spacy (Tagger),Spacy (Sentencizer)
Clean,0.754371,0.853603,0.820934
Partial punctuation,0.485907,0.517829,0.249753
Partial case,0.761754,0.825119,0.819679
Partial punctuation and case,0.443704,0.458619,0.249873
No punctuation and case,0.166273,0.180859,0.004633
